# Sharing_Variables

googlesource.com [blog](https://chromium.googlesource.com/external/github.com/tensorflow/tensorflow/+/r0.7/tensorflow/g3doc/how_tos/variable_scope/index.md)

# Fail to share variables

tf.Variable always creates a new one.

In [1]:
import tensorflow as tf
tf.reset_default_graph()

def my_op(x):
    W = tf.Variable(tf.random_normal([32, 10]), name="W")
    b = tf.Variable(tf.zeros([10]), name="b")
    return x @ W + b

image1 = tf.placeholder(tf.float32, shape=(None, 32), name='image1')
image2 = tf.placeholder(tf.float32, shape=(None, 32), name='image2')

result1 = my_op(image1)
print(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES))

result2 = my_op(image2)
print(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES))

[<tf.Variable 'W:0' shape=(32, 10) dtype=float32_ref>, <tf.Variable 'b:0' shape=(10,) dtype=float32_ref>]
[<tf.Variable 'W:0' shape=(32, 10) dtype=float32_ref>, <tf.Variable 'b:0' shape=(10,) dtype=float32_ref>, <tf.Variable 'W_1:0' shape=(32, 10) dtype=float32_ref>, <tf.Variable 'b_1:0' shape=(10,) dtype=float32_ref>]


# First Solution - variables_dict

A common way to share variables is to create them in a separate piece of code and pass them to functions that use them. 

While convenient, creating variables like above, outside of the code, breaks encapsulation:

- The code that builds the graph must document the names, types, and shapes of variables to create.
- When the code changes, the callers may have to create more, or less, or different variables.

https://chromium.googlesource.com/external/github.com/tensorflow/tensorflow/+/r0.7/tensorflow/g3doc/how_tos/variable_scope/index.md

In [1]:
import tensorflow as tf
tf.reset_default_graph()

variables_dict = {"weight": tf.Variable(tf.random_normal([32, 10]), name="W"), \
                  "bias": tf.Variable(tf.zeros([10]), name="b")}

def my_op(x, variables_dict):
    return x @ variables_dict['weight'] + variables_dict['bias']

image1 = tf.placeholder(tf.float32, shape=(None, 32), name='image1')
image2 = tf.placeholder(tf.float32, shape=(None, 32), name='image2')

result1 = my_op(image1, variables_dict)
print(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES))

result2 = my_op(image2, variables_dict)
print(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES))

[<tf.Variable 'W:0' shape=(32, 10) dtype=float32_ref>, <tf.Variable 'b:0' shape=(10,) dtype=float32_ref>]
[<tf.Variable 'W:0' shape=(32, 10) dtype=float32_ref>, <tf.Variable 'b:0' shape=(10,) dtype=float32_ref>]


# Second Solution - tf.get_variable

tf.Variable always creates a new one.

tf.get_variable either shares or creates a new one:

- Case 1: If there is no variable with given name (including prefix) used yet, it creates a new one.

- Case 2: If there is a variable with given name (including prefix) used before and if we declare to reuse, it uses or shares the old one. 

- Case 3: If there is a variable with given name (including prefix) used before but if we don't declare to reuse, it raises ValueError. 

In [ ]:
import tensorflow as tf
tf.reset_default_graph()

with tf.variable_scope("one"):
    a = tf.get_variable("v", shape=[1]) 
with tf.variable_scope("one", reuse=True):
    b = tf.get_variable("v", shape=[1]) 
    
init = tf.global_variables_initializer()
    
assert(a is b)  # Assertion is true; they are same objects
print(a)
print(b)

with tf.Session() as sess:
    init.run()
    print(sess.run([a, b]))

In [ ]:
import tensorflow as tf
tf.reset_default_graph()

with tf.variable_scope("one"):
    a = tf.get_variable("v", shape=[1]) 
    tf.get_variable_scope().reuse_variables()
    b = tf.get_variable("v", shape=[1]) 
    
init = tf.global_variables_initializer()
    
assert(a is b)  # Assertion is true; they are same objects
print(a)
print(b)

with tf.Session() as sess:
    init.run()
    print(sess.run([a, b]))

In [ ]:
import tensorflow as tf
tf.reset_default_graph()

with tf.variable_scope("one", reuse=tf.AUTO_REUSE):
    a = tf.get_variable("v", shape=[1]) 
    b = tf.get_variable("v", shape=[1]) 
    
init = tf.global_variables_initializer()
    
assert(a is b)  # Assertion is true; they are same objects
print(a)
print(b)

with tf.Session() as sess:
    init.run()
    print(sess.run([a, b]))